<a href="https://colab.research.google.com/github/lefitzpatrick/BIL_SA_workflow/blob/main/multi_linear_regression_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a ML model that uses multivariant linear regression to predict P, E, and R. This will be a simple version of the model used to incorperate into a simple workflow for alpha/beta testing (minimum viable product).

Developer: Lindsay Fitzpatrick

In [1]:
# Import libraries
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
# Mount my google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the data

We will use NWM streamflow for total monthly runoff and CFSR for total overlake precip and evaporation.

In [3]:
# specified lake[0] for testing but will later convert to a loop through all the lakes.
lake = ['MiHur','Erie','Ont','Sup']

In [4]:
dir = '/content/drive/MyDrive/BIL SA Project/Modeling/Data-driven Modeling/Input datasets/'

data_1 = pd.read_csv(dir+lake[0]+'_dataset.csv',sep=',')
data_1['Date'] = pd.to_datetime(data_1['Date'])
data_1.set_index('Date', inplace=True)
# Convert to a 2D numpy array
#x = np.array(data_1[['R(m)','P(m)','E(m)','R(m-1)','P(m-1)','E(m-1)']]) #this can specify certain variables to use
x = np.array(data_1) # set to read all 12 variables (P,E,R 0->3)
x.shape

(501, 12)

In [5]:
dir = '/content/drive/MyDrive/BIL SA Project/Modeling/Inventories/Data/'
data_2 = pd.read_csv(dir+'L2SWBM_1950-2022/'+lake[0]+'NBSC_analysis19502022_prior19001969_1m.csv',sep=',')
data_2['Date'] = pd.to_datetime(data_2[['Year', 'Month']].assign(Day=1))
data_2.set_index('Date', inplace=True)

By merging the datasets we can select the CNBS values that correspond with the monthly values of the input data

In [6]:
df = data_1.merge(data_2['Median'], on='Date')
df.head()
# Create a 1D numpy array
y = np.array(df['Median'])
y.shape

(501,)

Let's fit the model using split data; 80% train and 20% test (standard split).

In [7]:
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
model_type = LinearRegression()
model = model_type.fit(x_train,y_train)

This tests the 20% testing data using the model we fit above. It prints the R2.

In [8]:
y_pred = model.predict(x_test)
r_sq_2 = model.score(x_test,y_test)
print(r_sq_2)

0.8679999412976183


In [9]:
## need to save the model here to some sort of output so we can just load it in the future##
saved_model = 'linear_regression_model.pkl'
pickle.dump(model, open(saved_model, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(saved_model, 'rb'))

In [10]:
# Now that we saved the model, read in forecast values and run them through the model to output forecasted NBS
x_new = np.arange(72).reshape((-1, 12)) #create a random array that will represent P, E, R. this will be replaced with forecasted data.
y_new = loaded_model.predict(x_new)
print('Predicted NBS for '+lake[0])
print('Month 1:',y_new[0])
print('Month 2:',y_new[1])
print('Month 3:',y_new[2])
print('Month 4:',y_new[3])
print('Month 5:',y_new[4])
print('Month 6:',y_new[5])

Predicted NBS for MiHur
Month 1: 61.27352237643299
Month 2: 66.70349908239368
Month 3: 72.13347578835436
Month 4: 77.56345249431504
Month 5: 82.99342920027573
Month 6: 88.42340590623641


In [11]:
## the array can then be saved as a csv file to load into another script to show figures/statistics for easy interpretation ##